In [1]:
import torch
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
from deeplab_resnet import DeepLabv3_plus
from copy import deepcopy
import concurrent.futures
import time
import glob
import sys
%matplotlib inline

In [2]:
epoch = [0,1,3,4]
with open('val_v_5.json','r') as f:
    val_file = json.load(f)
val_file = json.loads(val_file)

In [3]:
def compute_dismap(dismap, bbox):
    x_min, y_min, x_max, y_max = bbox[:]

    # draw bounding box
    cv2.line(dismap, (x_min, y_min), (x_max, y_min), color=1, thickness=1)
    cv2.line(dismap, (x_min, y_min), (x_min, y_max), color=1, thickness=1)
    cv2.line(dismap, (x_max, y_max), (x_max, y_min), color=1, thickness=1)
    cv2.line(dismap, (x_max, y_max), (x_min, y_max), color=1, thickness=1)

    tmp = (dismap > 0).astype(np.uint8) # mark boundary
    tmp_ = deepcopy(tmp)

    fill_mask = np.ones((tmp.shape[0] + 2, tmp.shape[1] + 2)).astype(np.uint8)
    fill_mask[1:-1, 1:-1] = tmp_
    cv2.floodFill(tmp_, fill_mask, (int((x_min + x_max) / 2), int((y_min + y_max) / 2)), 5) # fill pixel inside bounding box

    tmp_ = tmp_.astype(np.int8)
    tmp_[tmp_ == 5] = -1 # pixel inside bounding box
    tmp_[tmp_ == 0] = 1 # pixel on and outside bounding box

    tmp = (tmp == 0).astype(np.uint8)

    dismap = cv2.distanceTransform(tmp, cv2.DIST_L2, cv2.DIST_MASK_PRECISE)  # compute distance inside and outside bounding box
    dismap = tmp_ * dismap + 128

    dismap[dismap > 255] = 255
    dismap[dismap < 0] = 0

    dismap = dismap.astype(np.uint8)

    return dismap

In [4]:
def distance_map(mask,bbox):
    bounds = (0, 0, mask.shape[1] - 1, mask.shape[0] - 1)


    x_min = max(bbox[0], bounds[0])
    y_min = max(bbox[1], bounds[1])
    x_max = min(bbox[2], bounds[2])
    y_max = min(bbox[3], bounds[3])

    bbox = [x_min, y_min, x_max, y_max]

    dismap = np.zeros((mask.shape[0], mask.shape[1]))
    dismap = compute_dismap(dismap, bbox)
    return dismap

In [5]:
def model_load(model_name):
    net = DeepLabv3_plus(n_classes=1, nInputChannels=4)
    checkpoint = torch.load(model_name, map_location=lambda storage, loc: storage)
    pretrained_dict = {k: v for k, v in checkpoint.items() if k in net.state_dict()}
    model_dict = net.state_dict()
    model_dict.update(pretrained_dict)
    net.load_state_dict(model_dict)
    net.eval()
    return net

In [6]:
def save_prediction(Resume_flag, Resume_box,outputs,result_shape, mask_dir, img_path, thres=0.75):
    prediction = np.transpose(outputs.data.numpy()[0, ...], (1, 2, 0))
    prediction = 1 / (1 + np.exp(-prediction))
    prediction = np.squeeze(prediction)
    prediction[prediction>thres] = 255
    prediction[prediction<=thres] = 0
    prediction = prediction.astype(np.uint8)
    if Resume_flag:
        x_min_re, y_min_re, x_max_re, y_max_re = Resume_box
        h = y_max_re - y_min_re
        w = x_max_re - x_min_re
        temp = np.zeros((result_shape[0],result_shape[1]))
        prediction = cv2.resize(prediction, (w, h),interpolation=cv2.INTER_NEAREST)
        temp[y_min_re : y_max_re, x_min_re : x_max_re] = prediction
        prediction = temp
    else:
        prediction = cv2.resize(prediction, (result_shape[1], result_shape[0]),interpolation=cv2.INTER_NEAREST)
    result_path  = os.path.join(mask_dir, img_path.split('/')[-1].replace('.jpg', '.png'))
    cv2.imwrite(result_path, prediction)

In [7]:
def Refine_bbox(result_shape, bbox, img):
    x_min, y_min, x_max, y_max = bbox[:]
    mask_area = result_shape[0] * result_shape[1]
    h = y_max - y_min
    w = x_max - x_min
    instance_area = h * w
    if instance_area/mask_area < 0.4:
        y_border = result_shape[0]-1
        x_border = result_shape[1]-1
        y_min_re = max(0,y_min-h)
        y_max_re = min(y_max+h, y_border)
        x_min_re = max(0,x_min-w)
        x_max_re = min(x_max+w, x_border)
        img_re = img[y_min_re : y_max_re, x_min_re : x_max_re]
        return True,[x_min_re, y_min_re, x_max_re, y_max_re],[x_min-x_min_re, y_min-y_min_re, x_max - x_min_re, y_max - y_min_re], img_re
    else:
        return False,[1,1,1,1],bbox, img

In [8]:
def input_prepare(img_re, bbox_re):
    dis_map = distance_map(img_re, bbox_re)
    dis_map = cv2.resize(dis_map, (450, 450), interpolation=cv2.INTER_NEAREST)
    dis_map = dis_map[:,:, np.newaxis]
    res = cv2.resize(img_re, (450,450), interpolation=cv2.INTER_CUBIC)
    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
    res = np.concatenate((res, dis_map), axis=2)
    res = res.transpose((2,0,1))
    res = res[np.newaxis,:,:,:]
    res = torch.from_numpy(res).float()
    return res

In [9]:
def get_val_result_with_scaler(sample):
    select_model_path, key, category = sample
    net = model_load(select_model_path)
    print('models have been loaded!')
    base_dir = '/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_scaler_v_5/'
    if not os.path.exists(base_dir):
        os.mkdir(base_dir)
        print('/home/ningxinL/DeepGrabCut_Davis_val/result/ds_val_finetune_scaler_v_5/ make successfully!')
    result_dir = os.path.join(base_dir, key)
    if not os.path.exists(result_dir):
        os.mkdir(result_dir)
    temp = val_file[key]
    for frame in temp:
        img_path = temp[frame]['img_path']
        img = cv2.imread(img_path)
        result_shape = img.shape
        mask_area = result_shape[0] * result_shape[1]
        if category in temp[frame].keys():
            mask_dir = os.path.join(result_dir, category)
            if not os.path.exists(mask_dir):
                os.mkdir(mask_dir)
            bbox = temp[frame][category]['bbox']
            Resume_flag, Resume_box, bbox_re, img_re = Refine_bbox(result_shape, bbox, img)
            res = input_prepare(img_re, bbox_re)
            outputs = net.forward(res)
            outputs = outputs.to(torch.device('cpu'))
            save_prediction(Resume_flag, Resume_box,outputs,result_shape, mask_dir, img_path, thres=0.75)

In [10]:
num = 0
base_dir = '/home/ningxinL/model_zoo/DeepGrabCut/finetune_models_with_scaler/'
sample_list = []
for root,directions, files in os.walk(base_dir):
    for direction in directions:
        try:
            category, instance_num = direction.split('_')
        except:
            continue
        num +=1
        model_list = glob.glob(os.path.join(base_dir, direction)+'/*.pth')
        max_epoch_num = 0
        for model_path in model_list:
            epoch_num = int((model_path.split('-')[-1]).split('_')[0])
            if epoch_num > max_epoch_num:
                max_epoch_num = epoch_num
                select_model_path = model_path
        sample = [select_model_path, category, instance_num]
        sample_list.append(sample)

In [37]:
sample_list

[['/home/ningxinL/model_zoo/DeepGrabCut/finetune_models_with_scaler/india_1/deepgc_epoch-11_davis_finetune.pth',
  'india',
  '1'],
 ['/home/ningxinL/model_zoo/DeepGrabCut/finetune_models_with_scaler/gold-fish_3/deepgc_epoch-9_davis_finetune.pth',
  'gold-fish',
  '3'],
 ['/home/ningxinL/model_zoo/DeepGrabCut/finetune_models_with_scaler/drift-chicane_1/deepgc_epoch-11_davis_finetune.pth',
  'drift-chicane',
  '1'],
 ['/home/ningxinL/model_zoo/DeepGrabCut/finetune_models_with_scaler/gold-fish_2/deepgc_epoch-12_davis_finetune.pth',
  'gold-fish',
  '2'],
 ['/home/ningxinL/model_zoo/DeepGrabCut/finetune_models_with_scaler/india_2/deepgc_epoch-22_davis_finetune.pth',
  'india',
  '2'],
 ['/home/ningxinL/model_zoo/DeepGrabCut/finetune_models_with_scaler/gold-fish_5/deepgc_epoch-15_davis_finetune.pth',
  'gold-fish',
  '5'],
 ['/home/ningxinL/model_zoo/DeepGrabCut/finetune_models_with_scaler/shooting_3/deepgc_epoch-5_davis_finetune.pth',
  'shooting',
  '3'],
 ['/home/ningxinL/model_zoo/Deep

In [11]:
with concurrent.futures.ProcessPoolExecutor() as executor:
        executor.map(get_val_result_with_scaler, sample_list)

Constructing DeepLabv3+ model...
Number of classes: 1
Constructing DeepLabv3+ model...
Output stride: 16
Number of Input Channels: 4
Number of classes: 1
Output stride: 16
Constructing DeepLabv3+ model...
Number of Input Channels: 4
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Number of classes: 1
Number of classes: 1
Output stride: 16
Number of classes: 1
Constructing DeepLabv3+ model...
Output stride: 16
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Output stride: 16
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Number of Input Channels: 4
Number of Input Channels: 4
Number of Input Channels: 4
Number of classes: 1
Number of classes: 1
Number of classes: 1
Constructing DeepLabv3+ model...
Constructing DeepLabv3+ model...
Number of classes: 1
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Output stride: 16
Number of classes: 1
Output s

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!
models have been loaded!
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!
models have been loaded!
models have been loaded!


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4


...

Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!
models have been loaded!
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


Constructing DeepLabv3+ model...
Number of classes: 1
Constructing DeepLabv3+ model...
Output stride: 16
Number of classes: 1
Number of Input Channels: 4
Output stride: 16
Number of Input Channels: 4
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!
models have been loaded!
models have been loaded!


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!
models have been loaded!
models have been loaded!
models have been loaded!
Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!


In [48]:
get_val_result_with_scaler(sample_list[0])

Constructing DeepLabv3+ model...
Number of classes: 1
Output stride: 16
Number of Input Channels: 4
models have been loaded!


KeyboardInterrupt: 

In [49]:
sample_list[0]

['/home/ningxinL/model_zoo/DeepGrabCut/finetune_models_with_scaler/india_1/deepgc_epoch-11_davis_finetune.pth',
 'india',
 '1']